In [2]:
# 載入需要的套件
import os
import numpy as np 
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# 設定 data_path
dir_data = './data/'
Train = os.path.join(dir_data, 'train.csv')
Test = os.path.join(dir_data, 'test.csv')

# 讀取檔案
Train_data = pd.read_csv(Train)
Test_data = pd.read_csv(Test)

In [54]:
Train_data.describe()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,3103.000000,1.393500e+04,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0,60000.000000,60000.0,60000.000000,6.000000e+04
mean,7.267917,11.414533,19488.957917,7.638250,1.931817,2.319200,11050.437183,1.585700,7.211837,5.025058e+04,...,60.867700,0.995800,252.823100,0.999867,5276.668033,1.0,16474.836600,1.0,101.044751,1.293727e+07
std,2.170741,4.751677,727.331019,5.070314,1.589972,1.399528,4344.018090,0.778952,20.453902,3.973488e+04,...,88.512972,0.064672,322.694074,0.011546,3856.219059,0.0,8998.020659,0.0,80.216061,5.522463e+07
min,1.000000,3.000000,18112.000000,1.000000,0.000000,0.000000,123.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,19.000000,1.0,30.000000,1.0,0.064766,2.261495e+05
25%,8.000000,7.000000,18892.750000,4.000000,0.000000,2.000000,7186.000000,2.000000,3.738412,2.827238e+04,...,20.000000,1.000000,88.000000,1.000000,2672.000000,1.0,8859.000000,1.0,49.182384,2.433114e+06
50%,8.000000,12.000000,19527.000000,5.000000,1.000000,2.000000,11507.000000,2.000000,5.758023,4.379195e+04,...,36.000000,1.000000,154.000000,1.000000,3942.000000,1.0,14344.000000,1.0,83.216237,5.240482e+06
75%,8.000000,13.000000,20125.000000,12.000000,4.000000,2.000000,14519.000000,2.000000,7.180619,6.148857e+04,...,64.000000,1.000000,275.000000,1.000000,6647.000000,1.0,23459.250000,1.0,130.779416,1.123932e+07
max,11.000000,21.000000,20665.000000,29.000000,4.000000,10.000000,20577.000000,2.000000,967.370868,1.020404e+06,...,1681.000000,1.000000,3463.000000,1.000000,18268.000000,1.0,32908.000000,1.0,1106.382478,5.088279e+09


In [55]:
Test_data.describe()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,500.000000,2290.000000,...,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.00000,10000.0,10000.000000,10000.0,10000.000000
mean,7.267800,11.408200,19490.418600,7.613600,1.935700,2.343600,11040.909900,1.58970,6.645918,50600.021883,...,14.862900,60.588400,0.995300,250.80290,0.999800,5217.24350,1.0,16330.445700,1.0,100.893000
std,2.178017,4.766408,729.350707,5.035755,1.585817,1.437409,4310.391506,0.77757,6.671072,38948.800177,...,27.498406,88.746682,0.068399,321.94302,0.014141,3816.06562,0.0,8956.747178,0.0,80.168798
min,1.000000,3.000000,18113.000000,1.000000,0.000000,0.000000,732.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,86.00000,1.0,402.000000,1.0,0.033160
25%,8.000000,7.000000,18894.000000,4.000000,0.000000,2.000000,7186.000000,2.00000,3.738412,28272.384431,...,4.000000,20.000000,1.000000,86.00000,1.000000,2658.00000,1.0,8767.000000,1.0,49.723905
50%,8.000000,12.000000,19524.000000,5.000000,1.000000,2.000000,11507.000000,2.00000,5.758023,43791.947141,...,8.000000,36.000000,1.000000,154.00000,1.000000,3898.00000,1.0,14209.500000,1.0,82.710367
75%,8.000000,13.000000,20124.000000,12.000000,4.000000,2.000000,14345.500000,2.00000,7.180619,61488.568950,...,16.000000,64.000000,1.000000,275.00000,1.000000,6601.25000,1.0,23210.000000,1.0,129.718101
max,11.000000,21.000000,20665.000000,29.000000,4.000000,10.000000,20516.000000,2.00000,103.895538,326960.895420,...,523.000000,1461.000000,1.000000,3177.00000,1.000000,18242.00000,1.0,32903.000000,1.0,1089.068319


In [4]:
train_Y = np.log1p(Train_data['total_price'])
ids = Test_data['building_id']
Train_data = Train_data.drop(['building_id', 'total_price'] , axis=1)
Test_data = Test_data.drop(['building_id'] , axis=1)
df = pd.concat([Train_data,Test_data])

df = df.fillna(-1)
df.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
0,8,21,18674,4,3,2,6271,2,-1.0,-1.000000,...,21,58,1,157,1,2483,1,6011,1,34.469803
1,8,7,18800,5,1,2,7885,2,-1.0,-1.000000,...,7,28,1,115,1,15872,1,32221,1,40.073573
2,8,7,19289,4,1,2,6028,2,-1.0,-1.000000,...,27,78,1,212,1,15760,1,32228,1,54.462081
3,8,21,20385,24,0,2,18325,0,-1.0,81138.889762,...,2,20,1,125,1,2568,1,7271,1,99.628966
4,1,21,20657,2,4,2,6880,2,-1.0,-1.000000,...,2,18,1,47,1,2587,1,7442,1,124.131236


In [5]:
train_num = train_Y.shape[0]
train_X = df[:train_num]

estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6442478810036997

In [6]:
estimator.fit(train_X, train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
test_X = df[train_num:]
predict = estimator.predict(test_X)

In [11]:
test_price = np.expm1(predict)

In [13]:
submit = pd.DataFrame(ids)
submit['total_price'] = test_price.astype(np.int32)
submit.to_csv("2019-05-07.csv",index=False)